## Wath is Numba

Numba is a library that allows translating Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. The complete and original guide can be faund at [https://nyu-cds.github.io/python-numba/](https://nyu-cds.github.io/python-numba/)

Numba provides the ability to speed up applications with high-performance functions written directly in Python, rather than using language extensions such as Cython.

With a few simple annotations, array-oriented and math-heavy Python code can be just-in-time (JIT) optimized to achieve performance similar to C, C++, and Fortran, without having to switch languages or Python interpreters.

Numba works at the function level. From a function, Numba can generate native code for that function as well as the wrapper code needed to call it directly from Python. This compilation is done on-the-fly and in-memory.

Numba’s main features are:

+ On-the-fly code generation (at import time or runtime, at the user’s preference)
+ Native code generation for the CPU (default) and GPU hardware
+ Integration with the Python scientific software stack (thanks to NumPy)

Numba's central feature is the ```numba.jit()``` decoration. Using this decorator, it is possible to mark a function for optimization by Numba’s JIT compiler. Various invocation modes trigger differing compilation options and behaviours.

Numba reads the Python bytecode for a decorated function and combines this with information about the types of the input arguments to the function. It analyzes and optimizes your code, and finally uses the LLVM compiler library to generate a machine code version of your function, tailored to your CPU capabilities. This compiled version is then used every time your function is called.

Let's see Numba in action. The following is a Python implementation of bubblesort for NumPy arrays.

In [ ]:
import os
import numpy as np
import math
os.environ["NUMBA_ENABLE_CUDASIM"] = "1"
from numba import jit, njit, vectorize, cuda



In [ ]:
original = np.arange(0.0, 10.0, 0.01, dtype='f4')
shuffled = original.copy()
np.random.shuffle(shuffled)

In [ ]:
def bubblesort(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [ ]:
sorted_arr = shuffled.copy()
bubblesort(sorted_arr)
print(np.array_equal(sorted_arr, original))

In [ ]:
%timeit sorted_arr[:] = shuffled[:]; bubblesort(sorted_arr)

In [ ]:
@cuda.jit
def bubblesort(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [ ]:
%timeit sorted_arr[:] = shuffled[:]; bubblesort(sorted_arr)

It is also possible to specify the signature of the Numba function. A function signature describes the types of the arguments and the return type of the function. This can produce slightly faster code as the compiler does not need to infer the types. However the function is no longer able to accept other types.

In the example below, ```float64(int32, int32)``` is the function's signature specifying a function that takes two 32-bit integer arguments and returns a double precision float. Numba provides a shorthand notation, so the same signature can be specified as ```f8(i4, i4)```.

The specialization will be compiled by the ```@jit decorator```, and no other specialization will be allowed. This is useful if you want fine-grained control over types chosen by the compiler (for example, to use single-precision floats).

If you omit the return type, e.g. by writing ```(int32, int32)``` instead of ```float64(int32, int32)```, Numba will try to infer it for you. Function signatures can also be strings, and you can pass several of them as a list; see the numba.jit() and cuda.jit() documentation for more details.

In [ ]:
from numba import jit, int32, float64

@jit(float64(int32, int32))
def f(x, y):
    # A somewhat trivial example
    return (x + y) / 3.14

In [ ]:
f(1, 3)

Numba has two compilation modes: ```nopython``` mode and ```object``` mode 

In ```nopython``` mode, the Numba compiler will generate code that does not access the Python C API. This mode produces the highest performance code, but requires that the native types of all values in the function can be inferred.

In ```object``` mode, the Numba compiler generates code that handles all values as Python objects and uses the Python C API to perform all operations on those objects. Code compiled in object mode will often run no faster than Python interpreted code.

In [ ]:
from numba import jit
@jit("void(f4[:])",nopython=True) #without simulator you can use cuda.jit
def bubblesort(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [ ]:
%timeit sorted_arr[:] = shuffled[:]; bubblesort(sorted_arr)

Numba's ```@vectorize``` decorator allows Python functions taking scalar input arguments to be used as NumPy ufuncs. Creating a traditional ```NumPy ufunc``` is not the most straightforward process and involves writing some C code. Numba makes this easy. Using the ```@vectorize``` decorator, Numba can compile a pure Python function into a ```ufunc``` that operates over NumPy arrays as fast as traditional ufuncs written in C.

The @vectorize decorator has two modes of operation:

+ *Eager* or decoration-time, compilation. If you pass one or more type signatures to the decorator, you will be building a Numpy ufunc. We’re just going to consider eager compilation here.
+ *Lazy*, or call-time, compilation. When not given any signatures, the decorator will give you a Numba dynamic universal function (DUFunc) that dynamically compiles a new kernel when called with a previously unsupported input type.

Using ```@vectorize```, you write your function as operating over input scalars, rather than arrays. Numba will generate the surrounding loop (or kernel) allowing efficient iteration over the actual inputs. 

The following code defines a function that takes two ```float32``` arrays and returns a ```float32``` array.

In [ ]:
import numpy as np
from numba import vectorize, int64, float32

@vectorize([float32(float32, float32)], target='parallel', fastmath=True) 
#to use cuda target must become cuda. Since emulator has a bug we use parallel
def vec_add(x, y):
    c =  x + y
    return c

a = np.random.rand(3,2).astype(np.float32) 
b = np.random.rand(3,2).astype(np.float32) 


print(vec_add(a, b))


### Writing CUDA kernels


CUDA has an execution model unlike the traditional sequential model used for programming CPUs. In CUDA, the code you write will be executed by multiple threads at once (often hundreds or thousands). Your solution will be modeled by defining a thread hierarchy of grid, blocks, and threads.

Numba also exposes three kinds of GPU memory:

+ global device memory
+ shared memory
+ local memory
For all but the simplest algorithms, it is important that you carefully consider how to use and access memory in order to minimize bandwidth requirements and contention.

NVIDIA recommends that programmers focus on following those recommendations to achieve the best performance:

- Find ways to parallelize sequential code
- Minimize data transfers between the host and the device
- Adjust kernel launch configuration to maximize device utilization
- Ensure global memory accesses are coalesced
- Minimize redundant accesses to global memory whenever possible
- Avoid different execution paths within the same warp

To execute a function on GPU, you have to either define something called a *kernel function* or a *device function*
+ kernels explicitly declare their thread hierarchy when called, i.e. the number of blocks and number of threads per block. You can compile your kernel once, and call it multiple times with different block and grid sizes.
+ kernels cannot return a value. So, either you will have to do changes on original array, or pass another array for storing the result. For computing scalar, you will have to pass a 1 element array.

In [ ]:
@cuda.jit
def increment_kernel(io_array):
    pos = cuda.grid(1)
    # x, y = cuda.grid(2) # For 2D array
    if pos < io_array.size:
        io_array[pos] += 1 # do the computation

In [ ]:
import numpy

# Create the data array - usually initialized some other way
data = numpy.ones(256)

# Set the number of threads in a block
threadsperblock = 32 

# Calculate the number of thread blocks in the grid
blockspergrid = (data.size + (threadsperblock - 1)) // threadsperblock

# Now start the kernel
increment_kernel[blockspergrid, threadsperblock](data)

# Print the result
print(data)

There are two main steps:

+ Instantiate the kernel proper, by specifying a number of blocks per grid and a number of threads per block. The product of the two will give the total number of threads launched. Kernel instantiation is done by taking the compiled kernel function and indexing it with a tuple of integers.
+ Running the kernel, by passing it the input array (and any separate output arrays if necessary). By default, running a kernel is synchronous: the function returns when the kernel has finished executing and the data is synchronized back.

The two-level thread hierarchy is important for the following reasons:

On the software side, the block size determines how many threads share a given area of shared memory.
On the hardware side, the block size must be large enough for full occupation of execution units; recommendations can be found in the CUDA C Programming Guide.
The block size you choose depends on a range of factors, including:

+ The size of the data array
+ The size of the shared mempory per block (e.g. 64KB)
+ The maximum number of threads per block supported by the hardware (e.g. 512 or 1024)
+ The maximum number of threads per multiprocessor (MP) (e.g. 2048)
+ The maximum number of blocks per MP (e.g. 32)
+ The number of threads that can be executed concurrently (a “warp” i.e. 32)
+ The execution of threads in a warp has a big effect on the computational throughput. If all threads in a warp are executing the same instruction then they can all be executed in parallel. But if one or more threads is executing a different instruction, the warp has to be split into groups of threads, and these groups execute serially.

Rules of thumb for threads per block:

+ Should be a round multiple of the warp size (32)
+ A good place to start is 128-512 but benchmarking is required to determine the optimal value.
+ Each streaming multiprocessor (SP) on the GPU must have enough active warps to achieve maximum throughput. In other words, the blocksize is usually selected to maximize the “occupancy”.


When running a kernel, the kernel function's code is executed by every thread once. It therefore has to know which thread it is in, in order to know which array element(s) it is responsible for. More complex algorithms may define more complex responsibilities, but the underlying principle is the same.

To help deal with multi-dimensional arrays, CUDA allows you to specify multi-dimensional blocks and grids. In the example above, you could make blockspergrid and threadsperblock tuples of one, two or three integers. Compared to 1-dimensional declarations of equivalent sizes, this doesn’t change anything to the efficiency or behaviour of generated code, but can help you write your algorithms in a more natural way.

One way is for the thread to determines its position in the grid and block and manually compute the corresponding array position:

In [ ]:
@cuda.jit
def double_kernel(io_array):
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
    # Block id in a 1D grid
    ty = cuda.blockIdx.x
    # Block width, i.e. number of threads per block
    bw = cuda.blockDim.x
    # Compute flattened index inside the array
    pos = tx + ty * bw
    if pos < io_array.size:  # Check array boundaries
        io_array[pos] *= 2 # do the computation
        
        
        
# Now start the kernel
double_kernel[blockspergrid, threadsperblock](data)

# Print the result
print(data)        

The following special objects are provided by the CUDA backend for the sole purpose of knowing the geometry of the thread hierarchy and the position of the current thread within that geometry:

+ numba.cuda.threadIdx - The thread indices in the current thread block. For 1-dimensional blocks, the index (given by the x attribute) is an integer spanning the range from 0 to numba.cuda.blockDim - 1. A similar rule exists for each dimension when more than one dimension is used.
+ numba.cuda.blockDim - The shape of the block of threads, as declared when instantiating the kernel. This value is the same for all threads in a given kernel, even if they belong to different blocks (i.e. each block is “full”).
+ numba.cuda.blockIdx - The block indices in the grid of threads launched a kernel. For a 1-dimensional grid, the index (given by the x attribute) is an integer spanning the range from 0 to numba.cuda.gridDim - 1. A similar rule exists for each dimension when more than one dimension is used.
+ numba.cuda.gridDim - The shape of the grid of blocks, i.e. the total number of blocks launched by this kernel invocation, as declared when instantiating the kernel.

These objects can be 1-, 2- or 3-dimensional, depending on how the kernel was invoked. To access the value at each dimension, use the x, y and z attributes of these objects, respectively.

In [ ]:
from numba import cuda
import numpy
import math

# CUDA kernel
@cuda.jit
def double_kernel_2D(io_array):
    row, col = cuda.grid(2)
    if row < io_array.shape[0] and col < io_array.shape[1]:
        io_array[row][col] *= 2 # do the computation

# Host code   
data = numpy.ones((1024,1024))

# Configure the blocks
threadsperblock = (64, 16) #ThreadsPerBlock must be a divisor of the data dimension: 1024/ 64 / 16 = 1
blockspergrid_x = int(math.ceil(data.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(data.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

double_kernel_2D[blockspergrid, threadsperblock](data)
print(data)

In [ ]:
#http://numba.pydata.org/numba-doc/0.13.4/CUDAJit.html

The *device function* are function that can be called only inside a kernel function. In other words they can be seen as *auxiliary functions*

In [ ]:
a = np.ones(100, dtype=np.float64)
b = np.ones(100, dtype=np.float64)

@cuda.jit(device=True)
def cudaDevice_func(a, b):
  for i in range(2):
    a = math.exp(a*b)
  return a

In [ ]:
@cuda.jit
def cudaKernal_func(a, b, result): # cuda.jit does not return result yet
  pos = cuda.grid(1)
  if (pos < a.shape[0]) and (pos < b.shape[0]):
    result[pos] = cudaDevice_func(a[pos], b[pos])

In [ ]:
result = np.zeros(100, dtype=np.float64)
threadsperblock = 32
blockspergrid = (a.shape[0] + (threadsperblock - 1)) // threadsperblock


cudaKernal_func[threadsperblock, blockspergrid](a, b, result)

result

At least you have to pay attention that CUDA kernel execution is designed to be asynchronous with respect to the host program. 
This means that the kernel launch (```double_kernel_2D[blockspergrid, threadsperblock](data)```) returns immediately, allowing the CPU to continue executing while the GPU works in the background. 
Only host<->device memory copies or an explicit synchronization call will force the CPU to wait until previously queued CUDA kernels are complete.

When you pass host NumPy arrays to a CUDA kernel, Numba has to synchronize on your behalf, but if you pass device arrays, processing will continue. If you launch multiple kernels in sequence without any synchronization in between, they will be queued up to run sequentially by the driver, which is usually what you want. If you want to run multiple kernels on the GPU in parallel (sometimes a good idea, but beware of race conditions!), take a look at CUDA streams.

In [ ]:
from numba import cuda
import numpy
import math

# CUDA kernel
@cuda.jit
def double_kernel_2D(io_array):
    row, col = cuda.grid(2)
    if row < io_array.shape[0] and col < io_array.shape[1]:
        io_array[row][col] *= 2 # do the computation

# Host code   
data = numpy.ones((1024,1024))
cuda.synchronize()
# Configure the blocks
threadsperblock = (64, 16) #ThreadsPerBlock must be a divisor of the data dimension: 1024/ 64 / 16 = 1
blockspergrid_x = int(math.ceil(data.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(data.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)


device_data = cuda.to_device(data)

double_kernel_2D[blockspergrid, threadsperblock](device_data)
cuda.synchronize()
print(device_data.copy_to_host())